In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 불러오기

In [2]:
DATA_PATH = "/content/drive/MyDrive/멋쟁이사자처럼/Final_Project/dataset/"
SEED = 42

In [3]:
import pandas as pd
import numpy as np

In [8]:
train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [9]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_1111.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_1111.csv")

train_ft.shape, test_ft.shape

((14940, 1470), (12225, 1470))

# 로그 변환 (Log Transformation):

- 주로 값의 범위가 넓을 때 사용하는 방법입니다. 값이 큰 쪽에 치우쳐 있거나 비대칭적인 분포를 가진 데이터를 변환해 분포를 균등하게 만듭니다.

- 예를 들어, 수입이나 소득처럼 큰 값들이 분포에 영향을 미치는 경우, 로그 변환을 하면 값의 스케일을 줄이고 분포를 더 대칭적으로 만들 수 있습니다.

- 사용법:
𝑥
x의 값이 0보다 커야 로그를 사용할 수 있습니다. 예를 들어, np.log(x) 또는 10을 밑으로 사용하고 싶다면 np.log10(x)를 사용합니다.

- 문제 정의: 데이터가 단순하고 예측력이 낮을 때.

- 비선형 변환:
    - log, sqrt, exp 등의 비선형 변환으로 데이터 스케일링.

In [10]:
# 수치형 피처만 필터링
numeric_df = train_ft.select_dtypes(include=['number'])

# 이상치 개수와 왜도를 저장할 딕셔너리
outlier_stats = {}

# 각 수치형 피처에서 이상치 개수와 왜도를 계산하여 딕셔너리에 저장
for column in numeric_df.columns:
    # IQR (Interquartile Range) 계산
    Q1 = numeric_df[column].quantile(0.25)
    Q3 = numeric_df[column].quantile(0.75)
    IQR = Q3 - Q1

    # 이상치의 기준점 설정 (IQR의 1.5배 초과/미만으로 간주)
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # 이상치 개수 계산
    outliers = numeric_df[(numeric_df[column] < lower_bound) | (numeric_df[column] > upper_bound)][column]
    outlier_count = len(outliers)

    # 왜도 계산
    skewness = numeric_df[column].skew()

    # 딕셔너리에 이상치 개수와 왜도 저장
    outlier_stats[column] = {'outlier_count': outlier_count, 'skewness': skewness}

# 이상치 개수가 747개 이상인 피처들만 필터링 --> 전체 데이터의 5% 이상
filtered_outliers = {k: v for k, v in outlier_stats.items() if v['outlier_count'] >= 747}

# 이상치 개수가 많은 피처를 내림차순으로 정렬
sorted_outliers = sorted(filtered_outliers.items(), key=lambda x: x[1]['outlier_count'], reverse=True)

# 결과 출력
for feature, stats in sorted_outliers:
    print(f"Feature: {feature}, Outlier Count: {stats['outlier_count']}, Skewness: {stats['skewness']:.2f}")

Feature: A202000지점_구매금액, Outlier Count: 3738, Skewness: 11.05
Feature: pv_대분류_여성정장_구매횟수, Outlier Count: 3704, Skewness: 9.54
Feature: pv_대분류_아동_스포츠_구입금액비중, Outlier Count: 3667, Skewness: 4.91
Feature: 대분류_여성캐쥬얼_구매주기, Outlier Count: 3655, Skewness: 2.74
Feature: pv_대분류_골프_유니캐쥬얼_구매횟수, Outlier Count: 3642, Skewness: 5.33
Feature: pv_대분류_골프_유니캐쥬얼_총구매금액, Outlier Count: 3586, Skewness: 11.53
Feature: pv_대분류_골프_유니캐쥬얼_평균구매금액, Outlier Count: 3586, Skewness: 5.70
Feature: pv_대분류_골프_유니캐쥬얼_구입금액비중, Outlier Count: 3586, Skewness: 4.30
Feature: pv_대분류_아동_스포츠_평균구매금액, Outlier Count: 3548, Skewness: 27.76
Feature: pv_대분류_여성정장_총구매금액, Outlier Count: 3542, Skewness: 10.41
Feature: pv_대분류_여성정장_평균구매금액, Outlier Count: 3542, Skewness: 9.11
Feature: pv_대분류_여성정장_구입금액비중, Outlier Count: 3542, Skewness: 3.83
Feature: pv_대분류_아동_스포츠_총구매금액, Outlier Count: 3532, Skewness: 14.10
Feature: 대분류_케주얼_구두_아동_구매주기, Outlier Count: 3451, Skewness: 3.15
Feature: pv_대분류_영플라자_구매횟수, Outlier Count: 3426, Skewness: 8.88
Feature: pv_대분류

In [7]:
# 이상치 개수가 747개 이상인 피처들만 필터링
filtered_outliers = {feature: stats for feature, stats in outlier_stats.items() if stats['outlier_count'] >= 747}

# 훈련 데이터에 대해 로그 변환
for feature in filtered_outliers:
    # 훈련 데이터에서 0 또는 음수 값 처리 후 로그 변환
    train_ft[feature] = train_ft[feature].apply(lambda x: np.log(x + 1e-9) if x >= 0 else np.log(np.abs(x) + 1e-9))

# 테스트 데이터에 대해서도 훈련 데이터에서 동일한 방식으로 로그 변환
for feature in filtered_outliers:
    # 테스트 데이터에 대해 훈련 데이터에서 계산된 방식대로 처리
    test_ft[feature] = test_ft[feature].apply(lambda x: np.log(x + 1e-9) if x >= 0 else np.log(np.abs(x) + 1e-9))

# 제곱근 변환 (Square Root Transformation):

- 값이 큰 범위에 몰려 있을 때 이를 줄여주기 위해 사용됩니다. 로그보다는 덜 극단적인 변환으로, 데이터의 분포를 완만하게 만듭니다.

- 예를 들어, 인구 수 같은 양수 데이터에서 제곱근을 적용하면 값의 분포를 균형 잡힌 형태로 만들 수 있습니다.

- 사용법: 양수 데이터를 대상으로 하며, np.sqrt(x)를 통해 제곱근을 적용할 수 있습니다.

## 제곱근 변환만 적용

In [ ]:
# 이상치 개수가 747개 이상인 피처들만 필터링
filtered_outliers = {feature: stats for feature, stats in outlier_stats.items() if stats['outlier_count'] >= 747}

# 훈련 데이터에 대해 제곱근 변환 적용
for feature in filtered_outliers:
    # 피처가 양수로만 이루어져 있는지 확인
    if (train_ft[feature] > 0).all():  # 모든 값이 양수일 때
        # 양수로만 이루어진 피처에는 제곱근 변환
        train_ft[feature] = train_ft[feature].apply(np.sqrt)
    else:
        # 0 또는 음수 값이 포함된 피처에도 제곱근 변환
        train_ft[feature] = train_ft[feature].apply(lambda x: np.sqrt(np.abs(x)) if x >= 0 else 0)

# 테스트 데이터에 대해서도 동일한 방식으로 제곱근 변환 적용
for feature in filtered_outliers:
    # 피처가 양수로만 이루어져 있는지 확인
    if (test_ft[feature] > 0).all():  # 모든 값이 양수일 때
        # 양수로만 이루어진 피처에는 제곱근 변환
        test_ft[feature] = test_ft[feature].apply(np.sqrt)
    else:
        # 0 또는 음수 값이 포함된 피처에도 제곱근 변환
        test_ft[feature] = test_ft[feature].apply(lambda x: np.sqrt(np.abs(x)) if x >= 0 else 0)


 ## 양수로만 이루어진 피처는 제곱근 이외는 로그변환

In [ ]:
# 이상치 개수가 747개 이상인 피처들만 필터링
filtered_outliers = {feature: stats for feature, stats in outlier_stats.items() if stats['outlier_count'] >= 747}

# 훈련 데이터에 대해 변환 적용
for feature in filtered_outliers:
    # 피처가 양수로만 이루어져 있는지 확인
    if (train_ft[feature] > 0).all():  # 모든 값이 양수일 때
        # 양수로만 이루어진 피처에는 제곱근 변환
        train_ft[feature] = train_ft[feature].apply(np.sqrt)
    else:
        # 0 또는 음수 값이 포함된 피처에는 로그 변환
        train_ft[feature] = train_ft[feature].apply(lambda x: np.log(x + 1e-9) if x >= 0 else np.log(np.abs(x) + 1e-9))

# 테스트 데이터에 대해서도 동일한 방식으로 변환 적용
for feature in filtered_outliers:
    # 피처가 양수로만 이루어져 있는지 확인
    if (test_ft[feature] > 0).all():  # 모든 값이 양수일 때
        # 양수로만 이루어진 피처에는 제곱근 변환
        test_ft[feature] = test_ft[feature].apply(np.sqrt)
    else:
        # 0 또는 음수 값이 포함된 피처에는 로그 변환
        test_ft[feature] = test_ft[feature].apply(lambda x: np.log(x + 1e-9) if x >= 0 else np.log(np.abs(x) + 1e-9))

## 피처 값 중 양수는 제곱근 변환 이외에는 로그 변환

In [11]:
# 이상치 개수가 747개 이상인 피처들만 필터링
filtered_outliers = {feature: stats for feature, stats in outlier_stats.items() if stats['outlier_count'] >= 747}

# 훈련 데이터에 대해 변환 적용
for feature in filtered_outliers:
    # 양수인 값은 제곱근 변환, 0 이하의 값은 로그 변환
    train_ft[feature] = train_ft[feature].apply(
        lambda x: np.sqrt(x) if x > 0 else np.log(x + 1e-9)  # 0 이하의 값은 로그 변환
    )
    # 변환 후 NaN 값이 있을 경우 0으로 대체
    train_ft[feature] = train_ft[feature].fillna(0)

# 테스트 데이터에 대해서도 동일한 방식으로 변환 적용
for feature in filtered_outliers:
    # 양수인 값은 제곱근 변환, 0 이하의 값은 로그 변환
    test_ft[feature] = test_ft[feature].apply(
        lambda x: np.sqrt(x) if x > 0 else np.log(x + 1e-9)  # 0 이하의 값은 로그 변환
    )
    # 변환 후 NaN 값이 있을 경우 0으로 대체
    test_ft[feature] = test_ft[feature].fillna(0)

<ipython-input-11-93a17ec4e807>:8: RuntimeWarning: invalid value encountered in log
  lambda x: np.sqrt(x) if x > 0 else np.log(x + 1e-9)  # 0 이하의 값은 로그 변환
<ipython-input-11-93a17ec4e807>:17: RuntimeWarning: invalid value encountered in log
  lambda x: np.sqrt(x) if x > 0 else np.log(x + 1e-9)  # 0 이하의 값은 로그 변환


# 지수 변환 (Exponential Transformation):

- 주로 값을 확장해야 하는 경우 사용합니다. 예를 들어, 분포가 너무 작게 몰려 있을 때 지수 변환을 통해 데이터의 차이를 크게 만들어 모델이 차이를 더 잘 학습하게 할 수 있습니다.

- 사용법: np.exp(x)로 변환합니다. 다만, 값이 너무 커질 수 있으니 주의가 필요합니다.

# Box-Cox 변환 (Box-Cox Transformation)

- 데이터 분포가 치우쳐져 있는 경우, 특히 정규 분포에 가깝게 만들기 위해 사용하는 변환 방법입니다.

- Box-Cox는 **람다(λ)**라는 파라미터를 사용해 최적의 변환을 찾습니다. 이 변환을 통해 데이터 분포를 정규에 가깝게 조정할 수 있습니다.

- 사용법: SciPy의 boxcox 함수를 사용하며, 데이터는 양수여야 합니다.

# 역변환

- 데이터의 값이 클수록 작은 값으로 줄어들게 만드는 방식으로, 비대칭 분포를 조정할 때 유용합니다.

- 사용법: 역수를 취해 변환하며, 1/x 형태로 사용할 수 있습니다.

# 피처 공학 및 스케일링

In [12]:
# ID 제거
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1469), (12225, 1469))

In [13]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
주구매_대분류,22
주구매_중분류,238


In [14]:
%pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 7.4 MB/s eta 0:00:00


In [15]:
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft["주구매지점"])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft["주구매지점"])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1473), (12225, 1473))

In [16]:
enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1475), (12225, 1475))

In [17]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1472), (12225, 1472))

In [18]:
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,평일방문비율,주말방문횟수,평일방문횟수,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,...,공휴일_대분류_영플라자_구매횟수,공휴일_대분류_잡화_구매횟수,공휴일_대분류_케주얼_구두_아동_구매횟수,공휴일_대분류_패션잡화_구매횟수,주구매지점_1,주구매지점_2,주구매지점_3,주구매지점_4,주구매_중분류_cnt,주구매_대분류_cnt
0,-0.255374,0.002987,0.257728,-0.257728,0.428635,-0.128919,0.346813,0.001191,0.838272,0.338186,...,-0.162079,-0.352780,-0.276145,-0.259675,1.654066,-0.623175,-0.612523,-0.468181,-0.741478,-1.315250
1,0.402970,-0.356452,-1.008554,1.008554,0.294145,0.554897,0.398608,-0.390607,0.620171,-0.552996,...,2.389509,-0.352780,-0.276145,3.890999,-0.604571,1.604686,-0.612523,-0.468181,-0.448069,-0.626416
2,1.911468,-0.869935,0.036742,-0.036742,0.718373,1.388325,0.410273,-0.514333,-0.304527,-0.059266,...,-0.162079,-0.352780,-0.276145,-0.259675,-0.604571,-0.623175,1.632592,-0.468181,-0.869072,1.064726
3,3.007893,-1.023980,-0.080558,0.080558,0.873482,2.358551,0.401120,-0.327474,0.008592,-0.135636,...,10.044273,-0.352780,3.562547,-0.259675,1.654066,-0.623175,-0.612523,-0.468181,1.515861,-0.248364
4,0.722054,-0.613193,0.302875,-0.302875,0.620560,0.657384,0.362380,1.707410,-0.130285,-0.821561,...,-0.162079,2.905522,3.691907,-0.259675,1.654066,-0.623175,-0.612523,-0.468181,-0.847712,1.064726


In [19]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [20]:
target = train_target["target"]

# 점수 확인

In [ ]:
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)

    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

# 이상치 기준 상위 500개 피처 로그변환 -> 0.7227222269087612
# 전체 데이터 중 이상치가 5% 이상인 피처에 대한 로그변환 -> 0.7227222269087612
# 제곱근 변환 -> 0.7209092345457006
# 양수로만 이뤄진 수치형 데이터는 제곱근 변환 + 나머지는 로그 변환 -> 0.7227222269087612
# 각 피처의 값 중 양수는 제곱근 변환, 나머지 0과 음수일 경우 로그 변환 -> 0.7234808173962948

Mean F1 Macro Score: 0.7234808173962948


----

In [ ]:
model = XGBClassifier(random_state=SEED, **params)
model.fit(train_ft, target)

In [ ]:
pred = model.predict(test_ft)
submit["target"] = pred
submit.to_csv("log_sqrt_transform.csv", index=False)

In [ ]:
train_ft.to_csv("train_common_v3.3_피처삭제X_log_sqrt.csv",index=False)
test_ft.to_csv("test_common_v3.3_피처삭제X_log_sqrt.csv",index=False)

# Optuna

In [ ]:
%pip install optuna

In [21]:
import optuna
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## XGBOOST

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
import xgboost as xgb
from xgboost import XGBClassifier

# 목적함수를 클래스로 만들기
class Objective:
    # 변수 설정
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = StratifiedKFold(5, shuffle= True, random_state= self.seed)

    def __call__(self, trial):  # 콜백함수 역할
        hp = {  "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "max_depth": trial.suggest_int("max_depth", 3, 10),
                "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
                "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                "gamma": trial.suggest_float("gamma", 0, 5)  }
        model = XGBClassifier(**hp, random_state= self.seed)
        score = cross_val_score(model, self.x, self.y, cv= self.cv, scoring= "f1_macro", n_jobs= -1).mean()
        return score

# Sampler 객체 생성(대체모델 역할)
sampler = optuna.samplers.TPESampler(seed= SEED)

# study 객체 생성
study = optuna.create_study(direction= "maximize", sampler= sampler)

# optimize 매서드 실행
objective_func = Objective(train_ft, target, SEED)
study.optimize(objective_func, n_trials= 100)

[I 2024-11-12 08:24:56,307] A new study created in memory with name: no-name-b533bcd4-7568-49f3-b0f1-564ac63a6a81
[I 2024-11-12 08:33:12,712] Trial 0 finished with value: 0.6886174135705478 and parameters: {'n_estimators': 400, 'learning_rate': 0.2536999076681772, 'max_depth': 8, 'min_child_weight': 6, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'gamma': 0.2904180608409973}. Best is trial 0 with value: 0.6886174135705478.
[I 2024-11-12 08:44:47,172] Trial 1 finished with value: 0.7115088283992492 and parameters: {'n_estimators': 900, 'learning_rate': 0.07725378389307355, 'max_depth': 8, 'min_child_weight': 1, 'subsample': 0.9849549260809971, 'colsample_bytree': 0.9162213204002109, 'gamma': 1.0616955533913808}. Best is trial 1 with value: 0.7115088283992492.
[I 2024-11-12 08:48:33,829] Trial 2 finished with value: 0.706431646271415 and parameters: {'n_estimators': 200, 'learning_rate': 0.018659959624904916, 'max_depth': 5, 'min_child_weight': 6, 'subsample':

In [ ]:
xgb_best_params = {'n_estimators': 900,
                   'learning_rate': 0.021734683976721573,
                   'max_depth': 6,
                   'min_child_weight': 5,
                   'subsample': 0.7039147285447092,
                   'colsample_bytree': 0.5823954097077503,
                   'gamma': 2.3107011128410493}
# 0.7231554675553863

In [ ]:
print(study.best_trial.value)
print(study.best_trial.params)

## LGBM

In [ ]:
from lightgbm import LGBMClassifier

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
def objective(trial):
    hp = {
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),               # 리프 노드의 수
        "max_depth": trial.suggest_int("max_depth", 3, 15),                   # 최대 트리 깊이
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),  # 학습률
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),      # 부스팅 단계 수
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),  # 최소 데이터 수
        "subsample": trial.suggest_float("subsample", 0.6, 1.0, step=0.1),    # 샘플링 비율
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1),  # 피처 샘플링 비율
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),  # L1 규제
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True) # L2 규제
    }

    model = LGBMClassifier(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED) # 대체 모델 역할은 sampler 객체

# study 객체
study = optuna.create_study(
    direction="maximize",
    sampler = sampler
)

study.optimize(objective, n_trials=50)   # optimze 메서드에 콜백 함수(objective) 전달

[I 2024-11-12 15:50:00,454] A new study created in memory with name: no-name-ad2e0a12-ea88-411c-8784-d3a8d3e953fb
[I 2024-11-12 15:54:48,972] Trial 0 finished with value: 0.7108780019097494 and parameters: {'num_leaves': 69, 'max_depth': 15, 'learning_rate': 0.1205712628744377, 'n_estimators': 600, 'min_child_samples': 19, 'subsample': 0.6, 'colsample_bytree': 0.6, 'reg_alpha': 0.6245760287469893, 'reg_lambda': 0.002570603566117598}. Best is trial 0 with value: 0.7108780019097494.
[I 2024-11-12 15:56:52,862] Trial 1 finished with value: 0.7039690030821333 and parameters: {'num_leaves': 112, 'max_depth': 3, 'learning_rate': 0.2708160864249968, 'n_estimators': 900, 'min_child_samples': 25, 'subsample': 0.6, 'colsample_bytree': 0.6, 'reg_alpha': 5.472429642032198e-06, 'reg_lambda': 0.00052821153945323}. Best is trial 0 with value: 0.7108780019097494.
[I 2024-11-12 15:57:57,053] Trial 2 finished with value: 0.7142086215730521 and parameters: {'num_leaves': 76, 'max_depth': 6, 'learning_rat

In [33]:
lgbm_best_params = {'num_leaves': 33,
                    'max_depth': 11,
                    'learning_rate': 0.01944354088862471,
                    'n_estimators': 800,
                    'min_child_samples': 65,
                    'subsample': 0.8,
                    'colsample_bytree': 0.8,
                    'reg_alpha': 3.6529269182968447e-07,
                    'reg_lambda': 3.3190277841665685e-07}

# 0.7252597292985927

In [34]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(**lgbm_best_params, random_state=SEED)
model.fit(train_ft, target)

pred = model.predict(test_ft)
submit["target"] = pred
submit.to_csv("log_sqrt_LGBM.csv", index=False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.284892 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 135303
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 1185
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [ ]:
# 최적의 하이퍼파라미터 출력
print(f"Best trial: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

## GBM

In [28]:
from sklearn.ensemble import GradientBoostingClassifier

# Objective function
def objective(trial):
    hp = {
        "max_depth": trial.suggest_int("max_depth", 3, 15),                       # 최대 트리 깊이
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True), # 학습률
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),    # 부스팅 단계 수
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 100),       # 최소 분할 샘플 수
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 100),         # 리프 노드의 최소 샘플 수
        "subsample": trial.suggest_float("subsample", 0.6, 1.0, step=0.1),         # 샘플링 비율
        "max_features": trial.suggest_float("max_features", 0.6, 1.0, step=0.1)    # 피처 샘플링 비율
    }

    model = GradientBoostingClassifier(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

# Sampler
sampler = optuna.samplers.TPESampler(seed=SEED)

# Study 객체 생성
study = optuna.create_study(
    direction="maximize",
    sampler=sampler
)

study.optimize(objective, n_trials=50)  # objective 함수 전달

[I 2024-11-13 17:25:26,804] A new study created in memory with name: no-name-ff1f8c55-e18b-463f-88d8-f81997b49852
[I 2024-11-13 18:01:40,565] Trial 0 finished with value: 0.6928428189461788 and parameters: {'max_depth': 7, 'learning_rate': 0.2536999076681772, 'n_estimators': 800, 'min_samples_split': 61, 'min_samples_leaf': 16, 'subsample': 0.6, 'max_features': 0.6}. Best is trial 0 with value: 0.6928428189461788.
[W 2024-11-13 19:43:33,136] Trial 1 failed with parameters: {'max_depth': 14, 'learning_rate': 0.07725378389307355, 'n_estimators': 800, 'min_samples_split': 4, 'min_samples_leaf': 97, 'subsample': 1.0, 'max_features': 0.7} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-28-2dbc0ce54c35>", line 16, in objective
    score = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_

KeyboardInterrupt: 

In [ ]:
# 최적의 하이퍼파라미터 출력
print(f"Best trial: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

- gradientboosting parameter

In [35]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier

params = {
    'n_estimators': 306,
    'learning_rate': 0.04741755706638415,
    'max_depth': 5,
    'subsample': 0.9055240250602022
}

scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = GradientBoostingClassifier(**params, random_state=SEED)
    model.fit(x_train, y_train)

    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7140562733938418


## Random Forest

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import optuna

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import optuna

# Objective function
def objective(trial):
    # Penalty를 'l2'로 고정
    penalty = "l2"

    # Solver 선택: 'l2' penalty와 호환되는 옵션 중에서 선택
    solver = trial.suggest_categorical("solver", ["liblinear", "lbfgs", "sag", "saga"])

    # 하이퍼파라미터 설정
    hp = {
        "C": trial.suggest_float("C", 1e-4, 10.0, log=True),      # 정규화 강도 조절
        "penalty": penalty,                                       # 정규화 방식 고정
        "solver": solver,                                         # 최적화 알고리즘
        "max_iter": trial.suggest_int("max_iter", 100, 1000, step=100)  # 반복 횟수 조절
    }

    # Logistic Regression 모델 정의
    model = LogisticRegression(**hp, random_state=SEED)

    # 교차 검증 점수 계산
    score = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

# Sampler 설정
sampler = optuna.samplers.TPESampler(seed=SEED)

# Study 객체 생성
study = optuna.create_study(
    direction="maximize",
    sampler=sampler
)

study.optimize(objective, n_trials=100)  # objective 함수 전달

[I 2024-11-13 15:17:50,891] A new study created in memory with name: no-name-a030b34a-677b-4c4d-a035-830150c80db4
[I 2024-11-13 15:18:05,549] Trial 0 finished with value: 0.7125872317816507 and parameters: {'solver': 'lbfgs', 'C': 0.0006026889128682511, 'max_iter': 200}. Best is trial 0 with value: 0.7125872317816507.
[I 2024-11-13 15:18:12,870] Trial 1 finished with value: 0.701861534546638 and parameters: {'solver': 'lbfgs', 'C': 0.00012674255898937226, 'max_iter': 1000}. Best is trial 0 with value: 0.7125872317816507.
[I 2024-11-13 15:18:42,646] Trial 2 finished with value: 0.7092006291369471 and parameters: {'solver': 'liblinear', 'C': 0.0033205591037519565, 'max_iter': 600}. Best is trial 0 with value: 0.7125872317816507.
[I 2024-11-13 15:22:25,156] Trial 3 finished with value: 0.70730700627386 and parameters: {'solver': 'sag', 'C': 0.0028888383623653178, 'max_iter': 400}. Best is trial 0 with value: 0.7125872317816507.
[I 2024-11-13 15:22:48,088] Trial 4 finished with value: 0.69

In [27]:
# 최적의 하이퍼파라미터 출력
print(f"Best trial: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

Best trial: 0.7206239583520162
Best hyperparameters: {'solver': 'liblinear', 'C': 0.00028042459288154386, 'max_iter': 300}


# Feature Selection

In [29]:
from sklearn.feature_selection import SelectKBest, SelectPercentile, SelectFromModel
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import f1_score
import optuna

In [23]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

- XGB로 Feature Selection

In [24]:
xgb = XGBClassifier(random_state=SEED) # 특성 선택을 하기 위한 모델
model = LogisticRegression(random_state=SEED) # 학습용 모델

fs_xgb = SelectFromModel(xgb) # 특성 선택에 사용하기 위한 모델 객체를 전달해줘야함.
x = fs_xgb.fit_transform(train_ft, target) # 특성 선택이 완료된 입력 데이터가 ndarray 로 반환
scores = cross_val_score(model, x, target, cv=cv, scoring="f1_macro", n_jobs=-1)
scores.mean()

0.7141481678146386

- RandomForest로 Feature Selection

In [25]:
rfc = RandomForestClassifier(random_state=SEED) # 특성 선택을 하기 위한 모델
model = LogisticRegression(random_state=SEED) # 학습용 모델

fs_rf = SelectFromModel(rfc) # 특성 선택에 사용하기 위한 모델 객체를 전달해줘야함.
x = fs_rf.fit_transform(train_ft, target) # 특성 선택이 완료된 입력 데이터가 ndarray 로 반환
scores = cross_val_score(model, x, target, cv=cv, scoring="f1_macro", n_jobs=-1)
scores.mean()

0.7144958607740429

In [26]:
xgb_best_cols = fs_xgb.get_feature_names_out()
len(xgb_best_cols)

565

In [27]:
rf_best_cols = fs_rf.get_feature_names_out()
len(rf_best_cols)

359

## Logistic Regression

### XGB로 Feature Selection

In [30]:
def objective(trial):
    hp = {
        "C": trial.suggest_float("C", 1e-4, 10.0, log=True),  # 규제 강도
        "penalty": "l2",  # L2 규제로 고정
        "solver": trial.suggest_categorical("solver", ["lbfgs", "liblinear", "sag", "saga"]),
        "max_iter": trial.suggest_int("max_iter", 50, 500, step=50)  # 반복 횟수
    }
    model = LogisticRegression(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft[xgb_best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED) # 대체 모델 역할은 sampler 객체

# study 객체
study = optuna.create_study(
    direction="maximize",
    sampler = sampler
)

study.optimize(objective, n_trials=100)   # optimze 메서드에 콜백 함수(objective) 전달

[I 2024-11-14 00:16:29,217] A new study created in memory with name: no-name-acdfa85f-4c84-4b69-bcae-3e829652bc9e
[I 2024-11-14 00:16:39,808] Trial 0 finished with value: 0.7196347947149497 and parameters: {'C': 0.0074593432857265485, 'solver': 'lbfgs', 'max_iter': 100}. Best is trial 0 with value: 0.7196347947149497.
[I 2024-11-14 00:16:46,764] Trial 1 finished with value: 0.7084429380948787 and parameters: {'C': 0.00019517224641449495, 'solver': 'lbfgs', 'max_iter': 500}. Best is trial 0 with value: 0.7196347947149497.
[I 2024-11-14 00:20:40,120] Trial 2 finished with value: 0.7152030049041113 and parameters: {'C': 1.452824663751602, 'solver': 'saga', 'max_iter': 300}. Best is trial 0 with value: 0.7196347947149497.
[I 2024-11-14 00:21:00,520] Trial 3 finished with value: 0.7193503480369696 and parameters: {'C': 0.01444525102276306, 'solver': 'liblinear', 'max_iter': 200}. Best is trial 0 with value: 0.7196347947149497.
[I 2024-11-14 00:21:08,739] Trial 4 finished with value: 0.71727

In [31]:
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

Best hyperparameters: {'C': 0.0008541909431453405, 'solver': 'liblinear', 'max_iter': 100}
Best F1 Score: 0.7269445777558114


In [32]:
log_xgb_params = {'C': 0.0008541909431453405, 'solver': 'liblinear', 'max_iter': 100}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리 (선택된 피처만 사용)
    x_train = train_ft.iloc[tri][xgb_best_cols]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][xgb_best_cols]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    logistic_model_xgb = LogisticRegression(**log_xgb_params, random_state=SEED)
    logistic_model_xgb.fit(x_train, y_train)

    # 모델 저장
    models.append(logistic_model_xgb)

    # 예측 및 F1 매크로 스코어 계산
    pred = logistic_model_xgb.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7269445777558114


In [ ]:
logistic_model_xgb = LogisticRegression(**log_xgb_params)
logistic_model_xgb.fit(train_ft[xgb_best_cols],target)

proba = logistic_model_xgb.predict_proba(test_ft[xgb_best_cols])[:, 1]

submit["target"] = proba
submit.to_csv("비선형_fs_xgb_logistic.csv",index=False)

### RandomForest로 Feature Selection

In [35]:
def objective(trial):
    hp = {
        "C": trial.suggest_float("C", 1e-4, 10.0, log=True),  # 규제 강도
        "penalty": "l2",  # L2 규제로 고정
        "solver": trial.suggest_categorical("solver", ["lbfgs", "liblinear", "sag", "saga"]),
        "max_iter": trial.suggest_int("max_iter", 50, 500, step=50)  # 반복 횟수
    }
    model = LogisticRegression(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft[rf_best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED) # 대체 모델 역할은 sampler 객체

# study 객체
study = optuna.create_study(
    direction="maximize",
    sampler = sampler
)

study.optimize(objective, n_trials=100)   # optimze 메서드에 콜백 함수(objective) 전달

[I 2024-11-14 01:01:18,974] A new study created in memory with name: no-name-515b385a-0dba-494c-a082-43fd83b57374
[I 2024-11-14 01:01:23,133] Trial 0 finished with value: 0.7156598321166794 and parameters: {'C': 0.0074593432857265485, 'solver': 'lbfgs', 'max_iter': 100}. Best is trial 0 with value: 0.7156598321166794.
[I 2024-11-14 01:01:25,241] Trial 1 finished with value: 0.7022563109343716 and parameters: {'C': 0.00019517224641449495, 'solver': 'lbfgs', 'max_iter': 500}. Best is trial 0 with value: 0.7156598321166794.
[I 2024-11-14 01:03:10,202] Trial 2 finished with value: 0.7150207965858136 and parameters: {'C': 1.452824663751602, 'solver': 'saga', 'max_iter': 300}. Best is trial 0 with value: 0.7156598321166794.
[I 2024-11-14 01:03:20,237] Trial 3 finished with value: 0.7158336618542883 and parameters: {'C': 0.01444525102276306, 'solver': 'liblinear', 'max_iter': 200}. Best is trial 3 with value: 0.7158336618542883.
[I 2024-11-14 01:03:22,602] Trial 4 finished with value: 0.71491

In [36]:
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

Best hyperparameters: {'C': 0.0007691284934455126, 'solver': 'liblinear', 'max_iter': 500}
Best F1 Score: 0.7231951407689357


In [37]:
log_rf_params = {'C': 0.0007691284934455126, 'solver': 'liblinear', 'max_iter': 500}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리 (선택된 피처만 사용)
    x_train = train_ft.iloc[tri][rf_best_cols]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][rf_best_cols]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    logistic_model_rf = LogisticRegression(**log_rf_params, random_state=SEED)
    logistic_model_rf.fit(x_train, y_train)

    # 모델 저장
    models.append(logistic_model_rf)

    # 예측 및 F1 매크로 스코어 계산
    pred = logistic_model_rf.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7231951407689357


In [ ]:
logistic_model_rf = LogisticRegression(**log_rf_params)
logistic_model_rf.fit(train_ft[rf_best_cols],target)

pred = logistic_model_rf.predict_proba(test_ft[rf_best_cols])[:, 1]

submit["target"] = pred
submit.to_csv("logistic_model_rf.csv",index=False)

## LDA

### XGB로 Feature Selection

In [44]:
def objective(trial):
    # Optuna가 선택할 수 있는 하이퍼파라미터 정의
    solver = trial.suggest_categorical('solver', ['svd', 'lsqr', 'eigen'])

    # 'lsqr'와 'eigen'에서만 shrinkage 하이퍼파라미터가 사용되므로 조건 추가
    if solver in ['lsqr', 'eigen']:
        shrinkage = trial.suggest_float('shrinkage', 0.0, 1.0)
    else:
        shrinkage = None

    # n_components 하이퍼파라미터: 차원 수를 1부터 X의 특성 개수까지
    n_components = trial.suggest_int('n_components', 1, min(train_ft[xgb_best_cols].shape[1], len(np.unique(target)) - 1))

    # LDA 모델 정의
    lda = LinearDiscriminantAnalysis(solver=solver, shrinkage=shrinkage, n_components=n_components)

    score = cross_val_score(lda, train_ft[xgb_best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

# Optuna study 설정 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2024-11-14 01:48:00,056] A new study created in memory with name: no-name-dd240045-1785-4635-84c2-f18e5c08e8e2
[I 2024-11-14 01:48:04,190] Trial 0 finished with value: 0.718338125457728 and parameters: {'solver': 'eigen', 'shrinkage': 0.7984532386917654, 'n_components': 1}. Best is trial 0 with value: 0.718338125457728.
[I 2024-11-14 01:48:07,598] Trial 1 finished with value: 0.7212203016594261 and parameters: {'solver': 'lsqr', 'shrinkage': 0.13149245831097756, 'n_components': 1}. Best is trial 1 with value: 0.7212203016594261.
[I 2024-11-14 01:48:18,344] Trial 2 finished with value: 0.7135873653162664 and parameters: {'solver': 'svd', 'n_components': 1}. Best is trial 1 with value: 0.7212203016594261.
[I 2024-11-14 01:48:22,436] Trial 3 finished with value: 0.7225623912002644 and parameters: {'solver': 'eigen', 'shrinkage': 0.2116174177047112, 'n_components': 1}. Best is trial 3 with value: 0.7225623912002644.
[I 2024-11-14 01:48:32,898] Trial 4 finished with value: 0.713587365316

In [45]:
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

Best hyperparameters: {'solver': 'eigen', 'shrinkage': 0.49858677421095554, 'n_components': 1}
Best F1 Score: 0.7255501185709449


In [46]:
lda_xgb_params = {'solver': 'eigen', 'shrinkage': 0.49858677421095554, 'n_components': 1}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리 (선택된 피처만 사용)
    x_train = train_ft.iloc[tri][xgb_best_cols]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][xgb_best_cols]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    lda_model_xgb = LinearDiscriminantAnalysis(**lda_xgb_params)
    lda_model_xgb.fit(x_train, y_train)

    # 모델 저장
    models.append(lda_model_xgb)

    # 예측 및 F1 매크로 스코어 계산
    pred = lda_model_xgb.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7255501185709449


In [ ]:
lda_model_xgb = LinearDiscriminantAnalysis(**lda_xgb_params)
lda_model_xgb.fit(train_ft[xgb_best_cols],target)

proba = lda_model_xgb.predict_proba(test_ft[xgb_best_cols])[:, 1]

submit["target"] = pred
submit.to_csv("fs_xgb_LDA.csv",index=False)

### RandomForest로 Feature Selection

In [41]:
def objective(trial):
    # Optuna가 선택할 수 있는 하이퍼파라미터 정의
    solver = trial.suggest_categorical('solver', ['svd', 'lsqr', 'eigen'])

    # 'lsqr'와 'eigen'에서만 shrinkage 하이퍼파라미터가 사용되므로 조건 추가
    if solver in ['lsqr', 'eigen']:
        shrinkage = trial.suggest_float('shrinkage', 0.0, 1.0)
    else:
        shrinkage = None

    # n_components 하이퍼파라미터: 차원 수를 1부터 X의 특성 개수까지
    n_components = trial.suggest_int('n_components', 1, min(train_ft[rf_best_cols].shape[1], len(np.unique(target)) - 1))

    # LDA 모델 정의
    lda = LinearDiscriminantAnalysis(solver=solver, shrinkage=shrinkage, n_components=n_components)

    score = cross_val_score(lda, train_ft[rf_best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

# Optuna study 설정 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2024-11-14 01:39:54,508] A new study created in memory with name: no-name-0f557ec0-fdc1-4267-a70b-980b9c5d3ad1
[I 2024-11-14 01:39:56,759] Trial 0 finished with value: 0.719298614431015 and parameters: {'solver': 'eigen', 'shrinkage': 0.5643827666062647, 'n_components': 1}. Best is trial 0 with value: 0.719298614431015.
[I 2024-11-14 01:39:58,781] Trial 1 finished with value: 0.7131223568203546 and parameters: {'solver': 'eigen', 'shrinkage': 0.8027910075070062, 'n_components': 1}. Best is trial 0 with value: 0.719298614431015.
[I 2024-11-14 01:40:00,610] Trial 2 finished with value: 0.7147013496352463 and parameters: {'solver': 'lsqr', 'shrinkage': 0.7654009838947428, 'n_components': 1}. Best is trial 0 with value: 0.719298614431015.
[I 2024-11-14 01:40:02,667] Trial 3 finished with value: 0.7081461958582442 and parameters: {'solver': 'eigen', 'shrinkage': 0.8471081617033561, 'n_components': 1}. Best is trial 0 with value: 0.719298614431015.
[I 2024-11-14 01:40:07,852] Trial 4 fini

In [42]:
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

Best hyperparameters: {'solver': 'lsqr', 'shrinkage': 0.40291758052927384, 'n_components': 1}
Best F1 Score: 0.7224334253910599


In [43]:
lda_rf_params = {'solver': 'lsqr', 'shrinkage': 0.32247003082505904, 'n_components': 1}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리 (선택된 피처만 사용)
    x_train = train_ft.iloc[tri][rf_best_cols]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][rf_best_cols]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    lda_model_rf = LinearDiscriminantAnalysis(**lda_rf_params)
    lda_model_rf.fit(x_train, y_train)

    # 모델 저장
    models.append(lda_model_rf)

    # 예측 및 F1 매크로 스코어 계산
    pred = lda_model_rf.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7217555155064295


In [ ]:
lda_model_rf = LinearDiscriminantAnalysis(**lda_rf_params)
lda_model_rf.fit(train_ft[rf_best_cols],target)

pred = lda_model_rf.predict_proba(test_ft[rf_best_cols])[:, 1]

submit["target"] = pred
submit.to_csv("lda_model_rf.csv",index=False)

## XGBoost

### XGB로 Feature Selection

In [47]:
def objective(trial):
    hp = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5)
    }

    model = XGBClassifier(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft[xgb_best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED) # 대체 모델 역할은 sampler 객체

# study 객체
study = optuna.create_study(
    direction="maximize",
    sampler = sampler
)

study.optimize(objective, n_trials=50)   # optimze 메서드에 콜백 함수(objective) 전달

[I 2024-11-14 02:04:41,257] A new study created in memory with name: no-name-df07c577-7bde-4a5e-8ef2-da6ea288d2f3
[I 2024-11-14 02:06:52,051] Trial 0 finished with value: 0.6953607300466209 and parameters: {'n_estimators': 400, 'learning_rate': 0.2536999076681772, 'max_depth': 8, 'min_child_weight': 6, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'gamma': 0.2904180608409973}. Best is trial 0 with value: 0.6953607300466209.
[I 2024-11-14 02:10:52,464] Trial 1 finished with value: 0.7154317344236184 and parameters: {'n_estimators': 900, 'learning_rate': 0.07725378389307355, 'max_depth': 8, 'min_child_weight': 1, 'subsample': 0.9849549260809971, 'colsample_bytree': 0.9162213204002109, 'gamma': 1.0616955533913808}. Best is trial 1 with value: 0.7154317344236184.
[I 2024-11-14 02:11:55,363] Trial 2 finished with value: 0.7066844309805294 and parameters: {'n_estimators': 200, 'learning_rate': 0.018659959624904916, 'max_depth': 5, 'min_child_weight': 6, 'subsample'

In [48]:
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

Best hyperparameters: {'n_estimators': 900, 'learning_rate': 0.03525364948006408, 'max_depth': 6, 'min_child_weight': 6, 'subsample': 0.9700861694708173, 'colsample_bytree': 0.7297941645661414, 'gamma': 0.6528144116808078}
Best F1 Score: 0.725324972881959


In [50]:
xgb_xgb_params = {'n_estimators': 900,
                   'learning_rate': 0.03525364948006408,
                   'max_depth': 6,
                   'min_child_weight': 6,
                   'subsample': 0.9700861694708173,
                   'colsample_bytree': 0.7297941645661414,
                   'gamma': 0.6528144116808078}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리 (선택된 피처만 사용)
    x_train = train_ft.iloc[tri][xgb_best_cols]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][xgb_best_cols]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    xgb_model_xgb = XGBClassifier(**xgb_xgb_params, random_state=SEED)
    xgb_model_xgb.fit(x_train, y_train)

    # 모델 저장
    models.append(xgb_model_xgb)

    # 예측 및 F1 매크로 스코어 계산
    pred = xgb_model_xgb.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.725324972881959


In [ ]:
xgb_model_xgb = XGBClassifier(**xgb_xgb_params)
xgb_model_xgb.fit(train_ft[xgb_best_cols],target)

pred = xgb_model_xgb.predict_proba(test_ft[xgb_best_cols])[:, 1]

submit["target"] = pred
submit.to_csv("fs_xgb_xgb.csv",index=False)

### RandomForest로 Feature Selection

In [ ]:
def objective(trial):
    hp = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5)
    }

    model = XGBClassifier(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft[rf_best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED) # 대체 모델 역할은 sampler 객체

# study 객체
study = optuna.create_study(
    direction="maximize",
    sampler = sampler
)

study.optimize(objective, n_trials=100)   # optimze 메서드에 콜백 함수(objective) 전달

In [ ]:
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

In [ ]:
xgb_rf_params = {}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리 (선택된 피처만 사용)
    x_train = train_ft.iloc[tri][rf_best_cols]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][rf_best_cols]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    xgb_model_rf = XGBClassifier(**lda_rf_params)
    xgb_model_rf.fit(x_train, y_train)

    # 모델 저장
    models.append(xgb_model_rf)

    # 예측 및 F1 매크로 스코어 계산
    pred = xgb_model_rf.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

In [ ]:
xgb_model_rf = XGBClassifier(**log_rf_params)
xgb_model_rf.fit(train_ft[rf_best_cols],target)

pred = xgb_model_rf.predict_proba(test_ft[rf_best_cols])[:, 1]

submit["target"] = pred
submit.to_csv("fs_rf_xgb.csv",index=False)

## LGBM

### XGB로 Feature Selection

In [ ]:
def objective(trial):
    hp = {
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),               # 리프 노드의 수
        "max_depth": trial.suggest_int("max_depth", 3, 15),                   # 최대 트리 깊이
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),  # 학습률
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),      # 부스팅 단계 수
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),  # 최소 데이터 수
        "subsample": trial.suggest_float("subsample", 0.6, 1.0, step=0.1),    # 샘플링 비율
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1),  # 피처 샘플링 비율
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),  # L1 규제
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True) # L2 규제
    }

    model = LGBMClassifier(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft[xgb_best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED) # 대체 모델 역할은 sampler 객체

# study 객체
study = optuna.create_study(
    direction="maximize",
    sampler = sampler
)

study.optimize(objective, n_trials=50)   # optimze 메서드에 콜백 함수(objective) 전달

[I 2024-11-14 05:10:58,893] A new study created in memory with name: no-name-467a80f0-7f2c-4d8a-9b6d-ffadbe38b6c0
[I 2024-11-14 05:13:53,463] Trial 0 finished with value: 0.7117188649601597 and parameters: {'num_leaves': 69, 'max_depth': 15, 'learning_rate': 0.1205712628744377, 'n_estimators': 600, 'min_child_samples': 19, 'subsample': 0.6, 'colsample_bytree': 0.6, 'reg_alpha': 0.6245760287469893, 'reg_lambda': 0.002570603566117598}. Best is trial 0 with value: 0.7117188649601597.
[I 2024-11-14 05:14:54,541] Trial 1 finished with value: 0.7113114607818796 and parameters: {'num_leaves': 112, 'max_depth': 3, 'learning_rate': 0.2708160864249968, 'n_estimators': 900, 'min_child_samples': 25, 'subsample': 0.6, 'colsample_bytree': 0.6, 'reg_alpha': 5.472429642032198e-06, 'reg_lambda': 0.00052821153945323}. Best is trial 0 with value: 0.7117188649601597.
[I 2024-11-14 05:15:30,787] Trial 2 finished with value: 0.7186803403563937 and parameters: {'num_leaves': 76, 'max_depth': 6, 'learning_rat

In [ ]:
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

In [ ]:
xgb_lgbm_params = {}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리 (선택된 피처만 사용)
    x_train = train_ft.iloc[tri][xgb_best_cols]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][xgb_best_cols]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    lgbm_model_xgb = LGBMClassifier(**xgb_lgbm_params, random_state=SEED)
    lgbm_model_xgb.fit(x_train, y_train)

    # 모델 저장
    models.append(lgbm_model_xgb)

    # 예측 및 F1 매크로 스코어 계산
    pred = lgbm_model_xgb.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

In [ ]:
lgbm_model_xgb = LGBMClassifier(**xgb_xgb_params, random_state=SEED)
lgbm_model_xgb.fit(train_ft[xgb_best_cols],target)

pred = lgbm_model_xgb.predict_proba(test_ft[xgb_best_cols])[:, 1]

submit["target"] = pred
submit.to_csv("fs_xgb_lgbm.csv",index=False)

### RadomForest로 Feature Selection

In [ ]:
def objective(trial):
    hp = {
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),               # 리프 노드의 수
        "max_depth": trial.suggest_int("max_depth", 3, 15),                   # 최대 트리 깊이
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),  # 학습률
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),      # 부스팅 단계 수
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),  # 최소 데이터 수
        "subsample": trial.suggest_float("subsample", 0.6, 1.0, step=0.1),    # 샘플링 비율
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1),  # 피처 샘플링 비율
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),  # L1 규제
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True) # L2 규제
    }

    model = LGBMClassifier(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft[rf_best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED) # 대체 모델 역할은 sampler 객체

# study 객체
study = optuna.create_study(
    direction="maximize",
    sampler = sampler
)

study.optimize(objective, n_trials=100)   # optimze 메서드에 콜백 함수(objective) 전달

In [ ]:
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

In [ ]:
rf_lgbm_params = {}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리 (선택된 피처만 사용)
    x_train = train_ft.iloc[tri][rf_best_cols]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][rf_best_cols]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    lgbm_model_rf = LGBMClassifier(**rf_lgbm_params, random_state=SEED)
    lgbm_model_rf.fit(x_train, y_train)

    # 모델 저장
    models.append(lgbm_model_rf)

    # 예측 및 F1 매크로 스코어 계산
    pred = lgbm_model_rf.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

In [ ]:
lgbm_model_rf = LGBMClassifier(**xgb_xgb_params, random_state=SEED)
lgbm_model_rf.fit(train_ft[rf_best_cols],target)

pred = lgbm_model_rf.predict_proba(test_ft[rf_best_cols])[:, 1]

submit["target"] = pred
submit.to_csv("fs_rf_lgbm.csv",index=False)